In [0]:
# ================================================
# SECURE UNITY CATALOG SETUP (SERVICE PRINCIPAL)
# ================================================
# This script uses a Service Principal ("Bot") to bypass
# Azure "Guest User" API limitations when creating
# Unity Catalog Storage Credentials.
#
# All sensitive credentials are fetched securely from
# Azure Key Vault via Databricks Secret Scopes.
# ================================================

import requests
import json

# --- 1. SECURE CONFIGURATION (Fetch from Key Vault) ---
# Scope Name: 'kv-secrets'
try:
    print("🔐 Fetching credentials from Key Vault...")
    client_id = dbutils.secrets.get(scope="kv-secrets", key="sp-client-id")
    tenant_id = dbutils.secrets.get(scope="kv-secrets", key="sp-tenant-id")
    client_secret = dbutils.secrets.get(scope="kv-secrets", key="sp-client-secret")
    # Resource ID isn't a secret, but good practice to keep config separate from code
    connector_id = dbutils.secrets.get(scope="kv-secrets", key="access-connector-id")
    print("✅ Credentials acquired successfully.")
except Exception as e:
    print("❌ Error fetching secrets. Ensure your Key Vault scope and secret names are correct.")
    raise e


# --- 2. GENERATE TOKEN AS THE BOT (Azure AD Login) ---
print("\n🤖 Bot is authenticating with Azure Active Directory...")
token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
token_data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "resource": "https://management.core.windows.net/"
}

token_resp = requests.post(token_url, data=token_data)

if token_resp.status_code != 200:
    # IMPORTANT: Do not print the response text here in production logs as it might leak info.
    # For debugging, it's okay.
    raise Exception(f"Bot Login Failed. Status: {token_resp.status_code}")
    
sp_mgmt_token = token_resp.json()["access_token"]
print("✅ Bot Azure Token Acquired!")


# --- 3. CALL DATABRICKS API AS THE BOT ---
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host_url = ctx.apiUrl().get()
db_token = ctx.apiToken().get()

# We pass BOTH tokens:
headers = {
    "Authorization": f"Bearer {db_token}",
    "X-Databricks-Azure-SP-Management-Token": sp_mgmt_token, 
    "Content-Type": "application/json"
}

payload = {
    "name": "cred_crypto_root",
    "azure_managed_identity": {
        "access_connector_id": connector_id
    },
    "comment": "Created via Service Principal Automation (Secure)"
}

print("\n🚀 Bot is attempting to create the Unity Catalog Credential...")
response = requests.post(
    f"{host_url}/api/2.1/unity-catalog/storage-credentials",
    headers=headers,
    json=payload
)

# --- 4. VALIDATE RESPONSE ---
if response.status_code == 200 or response.status_code == 201:
    print("✅ SUCCESS: Storage Credential 'cred_crypto_root' created!")
    print("You may now proceed with creating External Locations.")
elif response.status_code == 409:
    print("⚠️ ALREADY EXISTS: The credential 'cred_crypto_root' was already created.")
    print("You can proceed to the next step.")
else:
    print(f"❌ ERROR: Failed with status code {response.status_code}")
    print(f"Details: {response.text}")

🤖 Bot is authenticating with Azure...
✅ Bot Token Acquired!
🚀 Bot is creating the credential...
✅ SUCCESS: Storage Credential 'cred_crypto_root' created!


In [0]:
%sql
-- Replace <storage-account-name> with 'sacryptotradesdata'
CREATE EXTERNAL LOCATION IF NOT EXISTS loc_crypto_landing
    URL 'abfss://landing-zone@sacryptotradesdata.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `cred_crypto_root`)
    COMMENT 'Secure connection to the Landing Zone';

-- Verify it worked
DESCRIBE EXTERNAL LOCATION loc_crypto_landing;

name,url,credential_name,owner,created_by,created_at,comment
loc_crypto_landing,abfss://landing-zone@sacryptotradesdata.dfs.core.windows.net/,cred_crypto_root,raj.vinay2408@gmail.com,raj.vinay2408@gmail.com,2025-11-24T01:39:23.174Z,Secure connection to the Landing Zone


In [0]:
%sql
-- 1. Register ALL containers as External Locations
-- (We use the same credential 'cred_crypto_root' for all of them)

CREATE EXTERNAL LOCATION IF NOT EXISTS loc_crypto_bronze
    URL 'abfss://bronze@sacryptotradesdata.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `cred_crypto_root`);

CREATE EXTERNAL LOCATION IF NOT EXISTS loc_crypto_silver
    URL 'abfss://silver@sacryptotradesdata.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `cred_crypto_root`);

CREATE EXTERNAL LOCATION IF NOT EXISTS loc_crypto_gold
    URL 'abfss://gold@sacryptotradesdata.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `cred_crypto_root`);

CREATE EXTERNAL LOCATION IF NOT EXISTS loc_crypto_scripts
    URL 'abfss://scripts@sacryptotradesdata.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL `cred_crypto_root`);

In [0]:
%sql
-- Create the Catalog
CREATE CATALOG IF NOT EXISTS crypto_cat
    MANAGED LOCATION 'abfss://scripts@sacryptotradesdata.dfs.core.windows.net/catalog_meta'
    COMMENT 'The main catalog for the Crypto Trades Lakehouse';



In [0]:
%sql
-- Switch to this catalog for the next steps
USE CATALOG crypto_cat;

In [0]:
%sql
-- Create the Medallion Schemas
-- 1. Bronze Schema (Raw Data)
CREATE SCHEMA IF NOT EXISTS bronze
    MANAGED LOCATION 'abfss://bronze@sacryptotradesdata.dfs.core.windows.net/';

-- 2. Silver Schema (Cleaned/Masked Data)
CREATE SCHEMA IF NOT EXISTS silver
    MANAGED LOCATION 'abfss://silver@sacryptotradesdata.dfs.core.windows.net/';

-- 3. Gold Schema (Aggregated/Business Data)
CREATE SCHEMA IF NOT EXISTS gold
    MANAGED LOCATION 'abfss://gold@sacryptotradesdata.dfs.core.windows.net/';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS audit
    MANAGED LOCATION 'abfss://scripts@sacryptotradesdata.dfs.core.windows.net/';